Label Encoding застосовується для трьох бінарних змінних:
is_tv_subscriber
is_movie_package_subscriber
churn
Це перетворює ці змінні на числові значення (0 і 1), що полегшує їх використання в моделях машинного навчання.

Видалення стовпця id, який не є інформативним для моделювання, оскільки це лише ідентифікатор.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Загрузка данных из CSV файла
data = pd.read_csv('archive\internet_service_churn.csv')

# Кодування бінарних змінних
le = LabelEncoder()
data['is_tv_subscriber'] = le.fit_transform(data['is_tv_subscriber'])
data['is_movie_package_subscriber'] = le.fit_transform(data['is_movie_package_subscriber'])
data['churn'] = le.fit_transform(data['churn'])

# Видалення непотрібного стовпця 'id'
data = data.drop(columns=['id'])

# Перевіряємо результат кодування
print(data.head())

   is_tv_subscriber  is_movie_package_subscriber  subscription_age  bill_avg  \
0                 1                            0             11.95        25   
1                 0                            0              8.22         0   
2                 1                            0              8.91        16   
3                 0                            0              6.87        21   
4                 0                            0              6.39         0   

   reamining_contract  service_failure_count  download_avg  upload_avg  \
0                0.14                      0           8.4         2.3   
1                 NaN                      0           0.0         0.0   
2                0.00                      0          13.7         0.9   
3                 NaN                      1           0.0         0.0   
4                 NaN                      0           0.0         0.0   

   download_over_limit  churn  
0                    0      0  
1         